In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from mvgauss import MvGauss
from mixture import Mixture
from plot import plot_2d_contours

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8

In [ ]:
c0_pars = pd.DataFrame(index=('mu', 'sigma'))
c0_pars['a'] = 0, 1
c0_pars['b'] = -1, 1.5
c0_corr = {('a', 'b'): -0.8}
c0 = MvGauss.from_correlations(c0_pars, c0_corr)
c0

In [ ]:
c1_pars = pd.DataFrame(index=('mu', 'sigma'))
c1_pars['a'] = 0.5, 1
c1_pars['b'] = 3.0, 1.5
c1_corr = {('a', 'b'): 0.5}
c1 = MvGauss.from_correlations(c1_pars, c1_corr)

In [ ]:
mix = Mixture(models=[c0, c1],
              weights=[1, 2])
mix

In [ ]:
sample_size = 150
sample = mix.joint_sample(sample_size)

In [ ]:
ax = plt.gca()
ax.scatter(sample[:, 0], sample[:, 1],)
# ax.hexbin(sample[:,0],sample[:,1],)

In [ ]:
from sklearn.neighbors.kde import KernelDensity

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(sample)

In [ ]:
def kde_pdf(sample_coords):
    return np.exp(kde.score_samples(sample_coords))

In [ ]:
from sklearn import mixture

gmm_fixed_n = mixture.GaussianMixture(n_components=2)
gmm_fixed_n.fit(sample)

gmm_fixed_n.means_

In [ ]:
gmm_fixed_n.covariances_

In [ ]:
gmm_fixed_n.weights_

In [ ]:
def gmm_fixed_n_pdf(sample_coords):
    return np.exp(gmm_fixed_n.score_samples(sample_coords))

In [ ]:
n_components_max = 10
gmm_bayes = mixture.BayesianGaussianMixture(
    n_components=n_components_max,
    max_iter=1000,
    weight_concentration_prior=1. / (n_components_max**2)
)
gmm_bayes.fit(sample)
# np.set_printoptions(precision=3,suppress=True)

ax = plt.gca()
ax.bar(range(len(gmm_bayes.weights_)), gmm_bayes.weights_)
ax.set_title('Component weights in Gaussian mixture')
ax.set_xlabel('Component')
ax.set_ylabel('Weight')
gmm_bayes.converged_

In [ ]:
def gmm_bayes_pdf(sample_coords):
    return np.exp(gmm_bayes.score_samples(sample_coords))


gmm_bayes.weights_

In [ ]:
xlim=ylim=(-5,6)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(14, 8))
# for ax in axes.flat:
#     im = ax.imshow(np.random.random((10,10)), vmin=0, vmax=1)

levels = np.linspace(0., .085, num=10)
ax_idx = 0
plot_2d_contours(kde_pdf, xlim, ylim, ax=axes[ax_idx],
                 #                  levels=levels
                 )
axes[ax_idx].set_title('KDE')


ax_idx += 1
plot_2d_contours(gmm_fixed_n_pdf, xlim, ylim, ax=axes[ax_idx])
axes[ax_idx].set_title('GMM (fixed N)')

ax_idx += 1
plot_2d_contours(gmm_bayes_pdf, xlim, ylim, ax=axes[ax_idx])
axes[ax_idx].set_title('GMM (Bayes)')

ax_idx += 1
cset, _ = plot_2d_contours(mix.joint_pdf, xlim, ylim, ax=axes[ax_idx],)
# plt.colorbar(cset,ax=axes.tolist())
axes[ax_idx].set_title('True PDF')
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(im, cax=cbar_ax)